In [11]:
import pandas as pd
data = pd.read_excel('./data/성북구_행정동별_생활인구수.xlsx')
data_top5 = data.head(5)
targets = []
targets = data_top5['행정동명']
targets

0     종암동
1    길음1동
2     석관동
3     안암동
4     동선동
Name: 행정동명, dtype: object

In [31]:
import re
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup     
from selenium import webdriver
import time
import pandas as pd
import warnings

warnings.filterwarnings(action='ignore') 

options = webdriver.ChromeOptions()
options.add_argument("headless")
print('성북구 생활인구수 top5동 음식집 데이터 수집')
print()

for target in targets:
    print(target, '음식집(카카오맵) 데이터 수집 시작')

    url="https://map.kakao.com/"
    path = "./data/chromedriver.exe"
    driver = webdriver.Chrome(path)

    driver.get(url)
    time.sleep(0.5)

    searchbox = driver.find_element_by_xpath("//input[@id='search.keyword.query']")
    searchbox.send_keys(target + " 맛집")

    searchbutton = driver.find_element_by_xpath("//button[@id='search.keyword.submit']")
    driver.execute_script("arguments[0].click();", searchbutton)

    time.sleep(2)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    df = pd.DataFrame(columns=['업종','가게이름','평가점수','평가인원', '리뷰개수', '주소', "전화", "링크주소"])

    crawdata= soup.find_all("li","PlaceItem clickArea")

    for shop in range(len(crawdata)):

        li= soup.find_all("li","PlaceItem clickArea")[shop]
        sector = li.find("span","subcategory clickable").text 
        name = li.find("a","link_name").text               
        point = li.find("em","num").text                   
        person = li.find("a","numberofscore").text[:-1] 
        reviews2 = li.find("a","review").text[3:]         
        add1 = li.find("div","addr")
        add = add1.find_all("p")[0].text                    
        phone1 = li.find("div","contact clickArea")
        phone = phone1.find("span").text                         
        b_link = li.find_all("a")[7].get('href')

        df=df.append({'업종':sector,'가게이름':name,'평가점수':point,'평가인원':person, 
                      '리뷰개수':reviews2, '주소':add, "전화":phone, "링크주소":b_link}, ignore_index=True)

    print(target+" 1 번째 페이지 완료")

    # 더보기 버튼이 있을경우 더보기 버튼 누르기 (2번)
    try:
        time.sleep(1)
        element1 = driver.find_element_by_id("info.search.place.more")
        ActionChains(driver).move_to_element(element1).click(element1).perform()
        ActionChains(driver).move_to_element(element1).click(element1).perform()

        try:
            for nextpage in range(3,6,1):
                time.sleep(1)
                # 2페이지~5페이지
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                crawdata= soup.find_all("li","PlaceItem clickArea")[1]

                for shop in range(len(crawdata)):
                    selltime=""
                    li= soup.find_all("li","PlaceItem clickArea")[shop]
                    sector = li.find("span","subcategory clickable").text
                    name = li.find("a","link_name").text
                    point = li.find("em","num").text 
                    person = li.find("a","numberofscore").text[:-1] 
                    reviews2 = li.find("a","review").text[3:]
                    add1 = li.find("div","addr")
                    add = add1.find_all("p")[0].text
                    phone1 = li.find("div","contact clickArea")
                    phone = phone1.find("span").text
                    b_link = li.find_all("a")[7].get('href') 

                    df=df.append({'업종':sector,'가게이름':name,'평가점수':point,'평가인원':person, 
                                  '리뷰개수':reviews2, '주소':add, "전화":phone, "링크주소":b_link}, ignore_index=True)

                print( target, nextpage-1, "번째 페이지 완료")

                time.sleep(1)

                # 다음 페이지 버튼
                element1 = driver.find_element_by_id("info.search.page.no{}".format(nextpage))
                ActionChains(driver).move_to_element(element1).click(element1).perform()
        except:
            pass
        print(target+" 5 번째 페이지 완료")    

        time.sleep(1)
        
        # 다음 목록 >
        element1 = driver.find_element_by_id("info.search.page.next")
        ActionChains(driver).move_to_element(element1).click(element1).perform()

        # 6페이지~10페이지
        for nextpage in range(2,7,1):
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')

            crawdata= soup.find_all("li","PlaceItem clickArea")[1]
            
            for shop in range(len(crawdata)):
                selltime=""
                li= soup.find_all("li","PlaceItem clickArea")[shop]
                sector = li.find("span","subcategory clickable").text 
                name = li.find("a","link_name").text              
                point = li.find("em","num").text                
                person = li.find("a","numberofscore").text[:-1]    
                reviews2 = li.find("a","review").text[3:]     
                add1 = li.find("div","addr")
                add = add1.find_all("p")[0].text                   
                phone1 = li.find("div","contact clickArea")
                phone = phone1.find("span").text                  
                b_link = li.find_all("a")[7].get('href')

                df=df.append({'업종':sector,'가게이름':name,'평가점수':point,'평가인원':person, 
                              '리뷰개수':reviews2, '주소':add, "전화":phone, "링크주소":b_link}, ignore_index=True)

            print( target, nextpage+4, "번째 페이지 완료")
            time.sleep(1)

            # 다음 페이지 버튼
            element1 = driver.find_element_by_id("info.search.page.no{}".format(nextpage))
            ActionChains(driver).move_to_element(element1).click(element1).perform()
            
    except:
        pass

    savepath = "./data/"
    df.to_csv(savepath+target+" 음식집 카카오맵 데이터 크롤링.csv", index = False, mode='w', encoding = 'utf-8-sig')
    print(target+' 완료'+'================')
    print()
    driver.close()
    
print()
print('전체 데이터 수집 완료')

성북구 생활인구수 top5동 음식집 데이터 수집

종암동 음식집(카카오맵) 데이터 수집 시작
종암동 1 번째 페이지 완료
종암동 2 번째 페이지 완료
종암동 3 번째 페이지 완료
종암동 4 번째 페이지 완료
종암동 5 번째 페이지 완료
종암동 6 번째 페이지 완료
종암동 7 번째 페이지 완료
종암동 8 번째 페이지 완료
종암동 9 번째 페이지 완료
종암동 10 번째 페이지 완료
종암동 완료================

길음1동 음식집(카카오맵) 데이터 수집 시작
길음1동 1 번째 페이지 완료
길음1동 2 번째 페이지 완료
길음1동 3 번째 페이지 완료
길음1동 4 번째 페이지 완료
길음1동 5 번째 페이지 완료
길음1동 6 번째 페이지 완료
길음1동 7 번째 페이지 완료
길음1동 8 번째 페이지 완료
길음1동 9 번째 페이지 완료
길음1동 10 번째 페이지 완료
길음1동 완료================

석관동 음식집(카카오맵) 데이터 수집 시작
석관동 1 번째 페이지 완료
석관동 2 번째 페이지 완료
석관동 3 번째 페이지 완료
석관동 4 번째 페이지 완료
석관동 5 번째 페이지 완료
석관동 6 번째 페이지 완료
석관동 7 번째 페이지 완료
석관동 8 번째 페이지 완료
석관동 9 번째 페이지 완료
석관동 10 번째 페이지 완료
석관동 완료================

안암동 음식집(카카오맵) 데이터 수집 시작
안암동 1 번째 페이지 완료
안암동 2 번째 페이지 완료
안암동 3 번째 페이지 완료
안암동 4 번째 페이지 완료
안암동 5 번째 페이지 완료
안암동 6 번째 페이지 완료
안암동 7 번째 페이지 완료
안암동 8 번째 페이지 완료
안암동 9 번째 페이지 완료
안암동 10 번째 페이지 완료
안암동 완료================

동선동 음식집(카카오맵) 데이터 수집 시작
동선동 1 번째 페이지 완료
동선동 2 번째 페이지 완료
동선동 3 번째 페이지 완료
동선동 4 번째 페이지 완료
동선동 5 번째 페이지 완료
동선동 6 번째 페이지 완료
동선동 